In [ ]:
import torch

import torch.nn as nn
import torch.nn.functional as F

import torchvision
import torchvision.transforms as transforms

import torch.optim as optim
import torch.backends.cudnn as cudnn

In [ ]:
class ResidualBlock(nn.Module):
    
    '''
    This residual block is defined by He et al.

    @article{He2015,
	  author = {Kaiming He and Xiangyu Zhang and Shaoqing Ren and Jian Sun},
   	title = {Deep Residual Learning for Image Recognition},
  	journal = {arXiv preprint arXiv:1512.03385},
	  year = {2015}
    }
    '''
    
    def __init__(self, in_channels, out_channels, kernel_size, padding, stride):
        super(ResidualBlock, self).__init__()
        self.conv_res1 = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, 
                                   kernel_size=kernel_size, padding=padding, stride=stride, bias=False)
        self.conv_res1_bn = nn.BatchNorm2d(num_features=out_channels, momentum=0.9)
        self.conv_res2 = nn.Conv2d(in_channels=out_channels, out_channels=out_channels, 
                                   kernel_size=kernel_size, padding=padding, bias=False)
        self.conv_res2_bn = nn.BatchNorm2d(num_features=out_channels, momentum=0.9)

        if stride != 1:
            self.downsample = nn.Sequential(
                nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(num_features=out_channels, momentum=0.9))
        else:
            self.downsample = None

        self.relu = nn.ReLU(inplace=False)

    def forward(self, x):
        residual = x

        out = self.relu(self.conv_res1_bn(self.conv_res1(x)))
        out = self.conv_res2_bn(self.conv_res2(out))

        if self.downsample is not None:
            residual = self.downsample(residual)

        out = self.relu(out)
        out = out + residual
        return out

In [ ]:
class Net(nn.Module):

    # This is the 9 layer residual network.

    def __init__(self):
        super(Net, self).__init__()

        self.conv = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(num_features=64, momentum=0.9),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(num_features=128, momentum=0.9),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            ResidualBlock(in_channels=128, out_channels=128, kernel_size=3, stride=1, padding=1),
            
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(num_features=256, momentum=0.9),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(num_features=256, momentum=0.9),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            ResidualBlock(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1),
            
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        self.fc = nn.Linear(in_features=1024, out_features=10, bias=True)

    def forward(self, x):
        out = self.conv(x)
        out = out.view(-1, out.shape[1] * out.shape[2] * out.shape[3])
        out = self.fc(out)
        return out

In [ ]:
from torchsummary import summary

model = Net().to("cpu")
summary(model, (3,32,32))

In [ ]:
# Starting with a batch size of 128
batch_size = 128

train_transform = transforms.Compose([transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(), transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),])

test_transform = transforms.Compose([transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),])

# Using the CIFAR-10 Dataset

training_set = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transform)
train_load = torch.utils.data.DataLoader(training_set, batch_size=batch_size, shuffle=True, num_workers=2)

test_set = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=test_transform)
test_load = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
for X, Y in train_load:
  print(X.shape)
  print(Y.shape)
  print(X[0])
  break

In [ ]:
# using Adam optimizer

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)

In [ ]:
train_acc = []
test_acc = []

In [ ]:
def train(data, model, loss_fn, optimizer):
  size = len(data.dataset)
  model.train()
  count = 0

  for batch, (X, y) in enumerate(data):
    X = X.to("cpu")
    y = y.to("cpu")

    pred = model(X)
    loss = loss_fn(pred, y)

    optimizer.zero_grad(set_to_none=True)
    loss.backward()  
    optimizer.step()

    count += (pred.argmax(1) == y).type(torch.float).sum().item()
    if batch % 100 == 0:
      loss = loss.item()
      current = len(X) * batch

      print(f"Data Loss: {loss:>6f} [{current:>5d}/{size:>5d}]")
  
  train_acc.append(count/size)

In [ ]:
def test(data, model, loss_fn):
  size = len(data.dataset)
  num_batches = len(data)
  model.eval()
  test_loss = 0
  count = 0
  
  with torch.no_grad():
    for X, y in data:
      X = X.to("cpu")
      y = y.to("cpu")

      pred = model(X)
      test_loss += loss_fn(pred, y).item()

      count += (pred.argmax(1) == y).type(torch.float).sum().item()
 
  test_loss /= num_batches
  count /= size
  test_acc.append(count)
  print(f"Accuracy: {(100*count):>0.01f}%, Average Loss: {test_loss:>7f}\n")

In [ ]:
epochs = 100
for i in range(epochs):
  print(f"Epoch {i+1}\n")
  train(train_load, model, loss_fn, optimizer)
  test(test_load, model, loss_fn)
  print("\n---------------------------------------")

In [ ]:
import matplotlib.pyplot as plt
x = [i for i in range(epochs)]
plt.plot(x,train_acc, label = 'train_acc')
plt.plot(x,test_acc, label = 'test_acc')
plt.legend()
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.title('Batch Size: 128')
plt.grid()
plt.show()